In [43]:
# from greem.utility.monitoring import BaseMonitoring, RegularMonitoring, PeriodicMonitoring
from codecarbon import OfflineEmissionsTracker
import os
import pandas as pd
from time import sleep

In [44]:
host_name: str = os.uname()[1]
host_name

'bauer.itec.aau.at'

In [45]:
low_res_video: str = 'beauty_low_res.mp4'
high_res_video: str = 'beauty_high_res.mp4'

os.system(
    f'ffmpeg -n -re -i Beauty_s000.265 -b:v:0 1000k -minrate 1000k -maxrate 1000k -bufsize 3000k -c:v:0 libx264 -filter:v:0 "scale=640:360" -preset medium -keyint_min 10 -g 10 {low_res_video}')
os.system(f'ffmpeg -n -re -i Beauty_s000.265 -c:v:0 libx264 -crf 15 -preset medium {high_res_video}')

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvi

0

In [46]:
measurement_interval = 0.25
repetitions = 10

flushed_measurements: list = []

tracker = OfflineEmissionsTracker(
    measure_power_secs=measurement_interval,
    project_name='test test',
    log_level='error',
    save_to_file=False,
    country_iso_code='AUT')

for i in range(repetitions):
    tracker._project_name = f'high_res_{measurement_interval}s_{(i + 1)}'
    tracker.start()
    os.system(f'ffmpeg -y -re -i {high_res_video} -c:v:0 libx264 -preset medium test.mp4')
    flushed_measurements.append(tracker._prepare_emissions_data(delta=True))

    sleep(0.5)
    tracker._prepare_emissions_data(delta=True)

# sleep(10)

for i in range(repetitions):
    tracker._project_name = f'low_res{measurement_interval}s_{(i + 1)}'
    os.system(f'ffmpeg -y -re -i {low_res_video} -c:v:0 libx264 -preset medium test.mp4')
    flushed_measurements.append(tracker._prepare_emissions_data(delta=True))
    sleep(0.5)
    tracker._prepare_emissions_data(delta=True)

tracker.stop()

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_5 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvi

2.893631900578419e-05

In [52]:
df = pd.DataFrame(flushed_measurements)
df.to_csv(f'video_scaling_emissions_{host_name}.csv')